# Описание

В этом ноутбуке обучаем Roberta на отзывах на фильмы, сохраняем обученную модель в google drive и тестируем модель на отзывах на фильмы.

In [ ]:
MAX_LEN = 128 

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.0 MB/s 
     |████████████████████████████████| 182 kB 56.1 MB/s 
     |████████████████████████████████| 7.6 MB 44.0 MB/s 


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.0 MB/s 
     |████████████████████████████████| 588.3 MB 17 kB/s 
     |████████████████████████████████| 1.7 MB 35.6 MB/s 
     |████████████████████████████████| 6.0 MB 46.6 MB/s 
     |████████████████████████████████| 439 kB 70.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
max_len = MAX_LEN
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.BinaryCrossentropy()

input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
output = roberta_model([input_ids,attention_masks])
output = output[1]
output = tf.keras.layers.Dense(1, activation='sigmoid')(output)
model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=opt, loss=loss, metrics=METRICS)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_1[0][0]',                
 el)                            thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/IMDB Dataset.csv')


df['is_positive'] = (df['sentiment'] == 'positive').astype(int)
df.drop(columns=['sentiment'], inplace=True)
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df

,DATA_COLUMN,LABEL_COLUMN
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
# урезаем отзывы
parameter_const = 2.2 # подобрали
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x[:round(MAX_LEN * parameter_const)])

In [ ]:
df

,DATA_COLUMN,LABEL_COLUMN
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]
df_negative = df[df['LABEL_COLUMN']==0]
# Для тестовой выборки берем последние 10% негативных отзывов и последние 10% позитивных отзывов
n_test = df_negative.shape[0] // 10 
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 10 
df_positive_test = df_positive.tail(n_test)
df_balanced_test = pd.concat([df_negative_test, df_positive_test])
X_test = df_balanced_test['DATA_COLUMN'].squeeze().values
y_test = df_balanced_test['LABEL_COLUMN'].squeeze().values
# Обучающая выборка
n_train_neg = df_negative.shape[0] // 40 
df_negative_train = df_negative.head(n_train_neg)
n_train_pos = df_positive.shape[0] // 40 
df_positive_train = df_positive.head(n_train_pos)
df_balanced_train = pd.concat([df_negative_train, df_positive_train])
X_train = df_balanced_train['DATA_COLUMN'].squeeze().values
y_train = df_balanced_train['LABEL_COLUMN'].squeeze().values

In [ ]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# код ниже нужен для проверки

In [ ]:
token_lens = []

for txt in X_train:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 88


In [ ]:
for txt in X_test:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 124


In [ ]:
def tokenize_roberta(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)

In [ ]:
history_2 = model.fit([train_input_ids,train_attention_masks], y_train, epochs=5, batch_size=30)

Epoch 1/5
42/42 [==============================] - 1692s 39s/step - loss: 0.6812 - accuracy: 0.5832 - precision: 0.6048 - recall: 0.4800
Epoch 2/5
42/42 [==============================] - 1600s 38s/step - loss: 0.4817 - accuracy: 0.7784 - precision: 0.7710 - recall: 0.7920
Epoch 3/5
42/42 [==============================] - 1604s 38s/step - loss: 0.3109 - accuracy: 0.8584 - precision: 0.8478 - recall: 0.8736
Epoch 4/5
42/42 [==============================] - 1598s 38s/step - loss: 0.2086 - accuracy: 0.9208 - precision: 0.9175 - recall: 0.9248
Epoch 5/5
42/42 [==============================] - 1589s 38s/step - loss: 0.1453 - accuracy: 0.9488 - precision: 0.9445 - recall: 0.9536


In [ ]:
result_roberta = model.predict([test_input_ids,test_attention_masks])

157/157 [==============================] - 1991s 13s/step


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

y_predicted_roberta = np.where(result_roberta > 0.5, 1, 0)

df_roberta_for_reviews_on_movies = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])
df_roberta_for_reviews_on_movies.loc['reviews on movies', 'accuracy'] = accuracy_score(y_test, y_predicted_roberta)
df_roberta_for_reviews_on_movies.loc['reviews on movies', 'precision'] = precision_score(y_test, y_predicted_roberta)
df_roberta_for_reviews_on_movies.loc['reviews on movies', 'recall'] = recall_score(y_test, y_predicted_roberta)
df_roberta_for_reviews_on_movies.loc['reviews on movies', 'f1_score'] =  f1_score(y_test, y_predicted_roberta)

In [ ]:
df_roberta_for_reviews_on_movies

,accuracy,precision,recall,f1_score
reviews on movies,0.8378,0.837665,0.838,0.837832


Сохраняем обученую модель в google drive:

In [ ]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/roberta_trained_on_imdb_21_december'
model.save(saved_model_path, include_optimizer=True) 